In [1]:
import pandas as pd
import numpy as np

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_rows', None)

In [2]:
# func 모음

# 1. 종목코드를 6자리로 통일. ex) 990 -> 000990
def padding(x):
    x = str(x)
    return x.rjust(6, "0")

# 2. 결산월 12월인 것만 선택
# : 전체 데이터 4963개 중에서 12월 결산인 데이터가 4836개이다. 나머지 결산월의 데이터가 전체의 2.5%로 데이터량이 작기도하고 결측치가 많아 제거
def func_yearin(x):
    if x[5:7] == '12':
        return True
    else:
        return False

# 3. 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

# 4. 연결, 개별재무제표 통합을 위해 연결재무 데이터 컬럼명 통일화 : "(IFRS연결)"제거
def func_replace_col1(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS연결)","")

# 5. 연결, 개별재무제표 통합을 위해 개별재무 데이터 컬럼명 통일화 : "(IFRS)"제거
def func_replace_col2(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS)","")

# 6. 2020년 raw data 제외
def func_rm_2020(x):
    if x[:4] == "2020":
        return False
    else:
        return True

# 7. True, False 반전
def func_not(x):
    return not x

# 연결재무재표 merge

In [3]:
df_연결_일반성 = pd.read_csv("./datasets/raw/연결일반.csv",encoding='cp949')
df_연결_성장성 = pd.read_csv("./datasets/raw/연결성장.csv",encoding='cp949')
df_연결_수익성 = pd.read_csv("./datasets/raw/연결수익.csv",encoding="cp949")
df_연결_안정성 = pd.read_csv("./datasets/raw/연결안정.csv",encoding='cp949')

df_재무_연결 = pd.merge(df_연결_일반성, df_연결_성장성, on=["회사명","거래소코드","회계년도"], how = 'inner')
df_재무_연결 = pd.merge(df_재무_연결, df_연결_수익성, on=["회사명","거래소코드","회계년도"], how = 'inner')
df_재무_연결 = pd.merge(df_재무_연결, df_연결_안정성, on=["회사명","거래소코드","회계년도"], how = 'inner')
df_재무_연결["거래소코드"] = df_재무_연결["거래소코드"].map(padding) # 거래소코드 padding
df_재무_연결["회계년도"] = df_재무_연결["회계년도"].map(change_year) # 년도만 표시

list_col = list() # 컬럼명에서 "IFRS연결", " ", "," 제거
for i in list(df_재무_연결.columns):
    list_col.append(func_replace_col1(i))
df_재무_연결.columns = list_col

## 중복되는 데이터 제거
df_재무_연결.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df_재무_연결.reset_index(drop=True, inplace=True)

df_재무_연결.to_csv("./datasets/재무_연결.csv")
df_재무_연결.shape

(20953, 32)

In [4]:
df_재무_연결.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20953 entries, 0 to 20952
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   회사명            20953 non-null  object 
 1   거래소코드          20953 non-null  object 
 2   회계년도           20953 non-null  int64  
 3   상장일            20953 non-null  object 
 4   자산             16140 non-null  float64
 5   당기순이익(손실)      20404 non-null  float64
 6   미처분이익잉여금(결손금)  20404 non-null  float64
 7   총자본증가율         16085 non-null  float64
 8   비유동자산증가율       16085 non-null  float64
 9   유동자산증가율        16085 non-null  float64
 10  자기자본증가율        16085 non-null  float64
 11  매출액증가율         16085 non-null  float64
 12  정상영업이익증가율      16085 non-null  float64
 13  순이익증가율         16085 non-null  float64
 14  총포괄이익증가율       16085 non-null  float64
 15  매출액순이익률        16085 non-null  float64
 16  총자본순이익률        16085 non-null  float64
 17  자기자본순이익률       16085 non-null  float64
 18  매출원가대매

# 개별재무제표 merge

In [5]:
df_개별_일반성 = pd.read_csv("./datasets/raw/개별일반.csv",encoding='cp949')
df_개별_성장성 = pd.read_csv("./datasets/raw/개별성장.csv",encoding='cp949')
df_개별_수익성 = pd.read_csv("./datasets/raw/개별수익.csv",encoding="cp949")
df_개별_안정성 = pd.read_csv("./datasets/raw/개별안정.csv",encoding='cp949')



df_재무_개별 = pd.merge(df_개별_일반성, df_개별_성장성, on=["회사명","거래소코드","회계년도"], how = 'inner')
df_재무_개별 = pd.merge(df_재무_개별, df_개별_수익성, on=["회사명","거래소코드","회계년도"], how = 'inner')
df_재무_개별 = pd.merge(df_재무_개별, df_개별_안정성, on=["회사명","거래소코드","회계년도"], how = 'inner')
df_재무_개별["거래소코드"] = df_재무_개별["거래소코드"].map(padding) # 거래소코드 padding
df_재무_개별["회계년도"] = df_재무_개별["회계년도"].map(change_year) # 년도만 표시

list_col = list() # 컬럼명에서 "IFRS개별", " ", "," 제거
for i in list(df_재무_개별.columns):
    list_col.append(func_replace_col1(i))
df_재무_개별.columns = list_col

# 중복되는 데이터 제거
df_재무_개별.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df_재무_개별.reset_index(drop=True, inplace=True)

df_재무_개별.to_csv("./datasets/재무_개별.csv")
df_재무_개별.shape


(20953, 32)

In [6]:
df_재무_개별.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20953 entries, 0 to 20952
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   회사명            20953 non-null  object 
 1   거래소코드          20953 non-null  object 
 2   회계년도           20953 non-null  int64  
 3   상장일            20953 non-null  object 
 4   자산             20404 non-null  float64
 5   당기순이익(손실)      20404 non-null  float64
 6   미처분이익잉여금(결손금)  20404 non-null  float64
 7   총자본증가율         20330 non-null  float64
 8   비유동자산증가율       20330 non-null  float64
 9   유동자산증가율        20330 non-null  float64
 10  자기자본증가율        20330 non-null  float64
 11  매출액증가율         20330 non-null  float64
 12  정상영업이익증가율      20330 non-null  float64
 13  순이익증가율         20330 non-null  float64
 14  총포괄이익증가율       20330 non-null  float64
 15  매출액순이익률        20330 non-null  float64
 16  총자본순이익률        20330 non-null  float64
 17  자기자본순이익률       20330 non-null  float64
 18  매출원가대매

## 연결재무 데이터 결측치

In [7]:
df_재무_연결.isna().sum()

회사명                 0
거래소코드               0
회계년도                0
상장일                 0
자산               4813
당기순이익(손실)         549
미처분이익잉여금(결손금)     549
총자본증가율           4868
비유동자산증가율         4868
유동자산증가율          4868
자기자본증가율          4868
매출액증가율           4868
정상영업이익증가율        4868
순이익증가율           4868
총포괄이익증가율         4868
매출액순이익률          4868
총자본순이익률          4868
자기자본순이익률         4868
매출원가대매출액비율       4868
광고선전비대매출액비율      4868
유보율              4868
비유동비율            4868
유동비율             4868
당좌비율             4868
현금비율             4868
부채비율             4868
유동부채비율           4868
차입금비율            4868
이자보상배율(이자비용)     4868
유보액대비율           4868
유보액대납입자본배율       4868
투자집중도            4868
dtype: int64

In [7]:
## 외국계 기업 제거

def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

print("제거할 외국계기업 데이터 수 : ", len(df_재무_연결[df_재무_연결["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
df_재무_연결 = df_재무_연결[df_재무_연결["거래소코드"].map(func_isnt_fcompany)]
df_재무_연결.shape # 외국계기업 데이터 204개 제거

제거할 외국계기업 데이터 수 :  204


(20749, 32)

In [8]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df_재무_연결["회사명"].unique()))
print("거래소코드수 : ", len(df_재무_연결["거래소코드"].unique()))

회사수 :  2487
거래소코드수 :  2515


In [9]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(df_재무_연결["회사명"].unique()):
    if len(df_재무_연결[df_재무_연결["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수", len(list_overlap_code), "개")
# list_overlap_code

거래소코드가 2개인 기업 수 29 개


In [11]:
# 거래소코드 2개인 기업 리스트
list_overlap_code

['(주)본느',
 '(주)세화피앤씨',
 '(주)신성이엔지',
 '(주)아이엘사이언스',
 '(주)알로이스',
 '(주)에스에이티',
 '(주)엔케이맥스',
 '(주)원바이오젠',
 '(주)원익피앤이',
 '(주)인산가',
 '(주)자비스',
 '(주)정다운',
 '(주)카이노스메드',
 '(주)티에스트릴리온',
 '(주)판타지오',
 '(주)포인트엔지니어링',
 '(주)한국비엔씨',
 '(주)휴럼',
 '나무기술(주)',
 '삼성물산(주)',
 '소프트캠프(주)',
 '씨아이에스(주)',
 '에스케이(주)',
 '에스케이오션플랜트(주)',
 '엠에프엠코리아(주)',
 '원텍(주)',
 '줌인터넷(주)',
 '케이엠제약(주)',
 '패션플랫폼(주)']

In [10]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df_재무_연결[df_재무_연결["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df_재무_연결["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df_재무_연결[df_재무_연결["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df_재무_연결 = pd.concat(dfs)
df_재무_연결.reset_index(drop=True, inplace=True)

In [13]:
# 회사명이 2개인 거래소코드 존재 확인 
list_overlap_code = list()
for i in list(df_재무_연결["거래소코드"].unique()):
    if len(df_재무_연결[df_재무_연결["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

['036420']

In [11]:
## 기업명 변경
df_재무_연결[df_재무_연결['거래소코드']=='036420']
df_재무_연결.loc[df_재무_연결['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [12]:
df_재무_연결.isna().sum()

회사명                 0
거래소코드               0
회계년도                0
상장일                 0
자산               4754
당기순이익(손실)         375
미처분이익잉여금(결손금)     375
총자본증가율           4800
비유동자산증가율         4800
유동자산증가율          4800
자기자본증가율          4800
매출액증가율           4800
정상영업이익증가율        4800
순이익증가율           4800
총포괄이익증가율         4800
매출액순이익률          4800
총자본순이익률          4800
자기자본순이익률         4800
매출원가대매출액비율       4800
광고선전비대매출액비율      4800
유보율              4800
비유동비율            4800
유동비율             4800
당좌비율             4800
현금비율             4800
부채비율             4800
유동부채비율           4800
차입금비율            4800
이자보상배율(이자비용)     4800
유보액대비율           4800
유보액대납입자본배율       4800
투자집중도            4800
dtype: int64

In [15]:
# df_재무_연결.isna().sum().to_dict()
na_1 = int(list(df_재무_연결.isna().sum().to_dict().values()).count(0))
na_2 = int(list(df_재무_연결.isna().sum().to_dict().values()).count(4800)) # 데이터 하나도 없는 컬럼!
na_3 = int(list(df_재무_연결.isna().sum().to_dict().values()).count(375)) # 데이터 하나도 없는 기업&회계년도 데이터
print("전체컬럼수 :", len(list(df_재무_연결.isna().sum().to_dict().values())))
print("1) 인덱스컬럼수 : ", na_1)
print("2) 데이터 하나도 없는 컬럼수 : ", na_2)
print("3) 데이터 하나도 없는 행의 컬럼수에서 위 2)번 컬럼수 제외 : ", na_3)
print(" -> 인덱스컬럼을 제외한 모든 컬럼에 결측치 존재")


# df_재무_연결.isna().sum().to_dict()

# 대표적으로 "배당성향" 컬럼이 재무df_재무_연결재무제표 데이터에 없다.
# 그 이유는 ts2000이 "한국거래소 배당성향 계산 기준"인 별도 재무제표 기준으로 계산하기 떄문.

# ts2000의 배당성향과 naver 기준 배당성향이 다른데 ts2000은 한국거래소 기준으로 별도 재무제표 기준으로 계산하고있어 그에 따른 수치이고 
# Naver는 금감원기준으로 재무df_재무_연결 재무제표 기준으로 계산하고 있어 두 곳의 값이 다른걸 확인했다. 따라서 우리는 한국거래소기준으로 배당성향을 사용함을 확인.
# 나머지 7개 컬럼도 이와 같은 이유로 개별재무제표에서만 계산되기에 재무df_재무_연결재무제표 데이터에서 누락됐을 것이라 예

전체컬럼수 : 32
1) 인덱스컬럼수 :  4
2) 데이터 하나도 없는 컬럼수 :  25
3) 데이터 하나도 없는 행의 컬럼수에서 위 2)번 컬럼수 제외 :  2
 -> 인덱스컬럼을 제외한 모든 컬럼에 결측치 존재


## 개별재무 데이터결측

In [16]:
print(df_재무_개별.isna().sum())

회사명                0
거래소코드              0
회계년도               0
상장일                0
자산               549
당기순이익(손실)        549
미처분이익잉여금(결손금)    549
총자본증가율           623
비유동자산증가율         623
유동자산증가율          623
자기자본증가율          623
매출액증가율           623
정상영업이익증가율        623
순이익증가율           623
총포괄이익증가율         623
매출액순이익률          623
총자본순이익률          623
자기자본순이익률         623
매출원가대매출액비율       623
광고선전비대매출액비율      623
유보율              623
비유동비율            623
유동비율             623
당좌비율             623
현금비율             623
부채비율             623
유동부채비율           623
차입금비율            623
이자보상배율(이자비용)     623
유보액대비율           623
유보액대납입자본배율       623
투자집중도            623
dtype: int64


In [17]:
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

print("제거할 외국계기업 데이터 수 : ", len(df_재무_개별[df_재무_개별["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
df_재무_개별 = df_재무_개별[df_재무_개별["거래소코드"].map(func_isnt_fcompany)]
df_재무_개별.shape # 외국계기업 데이터 204개 제거

제거할 외국계기업 데이터 수 :  204


(20749, 32)

In [18]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df_재무_개별["회사명"].unique()))
print("거래소코드수 : ", len(df_재무_개별["거래소코드"].unique()))

회사수 :  2487
거래소코드수 :  2515


In [19]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(df_재무_개별["회사명"].unique()):
    if len(df_재무_개별[df_재무_개별["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수", len(list_overlap_code), "개")
# list_overlap_code

거래소코드가 2개인 기업 수 29 개


In [20]:

# 거래소코드 2개인 기업 리스트
print(list_overlap_code)


['(주)본느', '(주)세화피앤씨', '(주)신성이엔지', '(주)아이엘사이언스', '(주)알로이스', '(주)에스에이티', '(주)엔케이맥스', '(주)원바이오젠', '(주)원익피앤이', '(주)인산가', '(주)자비스', '(주)정다운', '(주)카이노스메드', '(주)티에스트릴리온', '(주)판타지오', '(주)포인트엔지니어링', '(주)한국비엔씨', '(주)휴럼', '나무기술(주)', '삼성물산(주)', '소프트캠프(주)', '씨아이에스(주)', '에스케이(주)', '에스케이오션플랜트(주)', '엠에프엠코리아(주)', '원텍(주)', '줌인터넷(주)', '케이엠제약(주)', '패션플랫폼(주)']


In [21]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df_재무_개별[df_재무_개별["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df_재무_개별["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df_재무_개별[df_재무_개별["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df_재무_개별 = pd.concat(dfs)
df_재무_개별.reset_index(drop=True, inplace=True)





In [23]:
# 회사명이 2개인 거래소코드 존재 확인 - 엄슴
list_overlap_code = list()
for i in list(df_재무_개별["거래소코드"].unique()):
    if len(df_재무_개별[df_재무_개별["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

['036420']

In [24]:
# 기업명 변경
df_재무_개별[df_재무_개별['거래소코드']=='036420']
df_재무_개별.loc[df_재무_개별['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [25]:
df_재무_개별.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20649 entries, 0 to 20648
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   회사명            20649 non-null  object 
 1   거래소코드          20649 non-null  object 
 2   회계년도           20649 non-null  int64  
 3   상장일            20649 non-null  object 
 4   자산             20274 non-null  float64
 5   당기순이익(손실)      20274 non-null  float64
 6   미처분이익잉여금(결손금)  20274 non-null  float64
 7   총자본증가율         20201 non-null  float64
 8   비유동자산증가율       20201 non-null  float64
 9   유동자산증가율        20201 non-null  float64
 10  자기자본증가율        20201 non-null  float64
 11  매출액증가율         20201 non-null  float64
 12  정상영업이익증가율      20201 non-null  float64
 13  순이익증가율         20201 non-null  float64
 14  총포괄이익증가율       20201 non-null  float64
 15  매출액순이익률        20201 non-null  float64
 16  총자본순이익률        20201 non-null  float64
 17  자기자본순이익률       20201 non-null  float64
 18  매출원가대매

## 개별 연결  merger
### 연결재무데이터의 결측치를 개별재무데이터로 결측치 대체

In [26]:
# df_재무를 완전데이터와 결측데이터로 나눈 뒤 결측데이터를 개별재무데이터로 대체시키고, df_재무로 다시 통합하면 끝!
df_재무_완전 =df_재무_연결[df_재무_연결["총자본순이익률"].isna().map(func_not)]
# df_재무_완전

df_재무_결측 =df_재무_연결[df_재무_연결["총자본순이익률"].isna()]
# df_재무_결측

In [27]:
df_재무_개별.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20649 entries, 0 to 20648
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   회사명            20649 non-null  object 
 1   거래소코드          20649 non-null  object 
 2   회계년도           20649 non-null  int64  
 3   상장일            20649 non-null  object 
 4   자산             20274 non-null  float64
 5   당기순이익(손실)      20274 non-null  float64
 6   미처분이익잉여금(결손금)  20274 non-null  float64
 7   총자본증가율         20201 non-null  float64
 8   비유동자산증가율       20201 non-null  float64
 9   유동자산증가율        20201 non-null  float64
 10  자기자본증가율        20201 non-null  float64
 11  매출액증가율         20201 non-null  float64
 12  정상영업이익증가율      20201 non-null  float64
 13  순이익증가율         20201 non-null  float64
 14  총포괄이익증가율       20201 non-null  float64
 15  매출액순이익률        20201 non-null  float64
 16  총자본순이익률        20201 non-null  float64
 17  자기자본순이익률       20201 non-null  float64
 18  매출원가대매

In [28]:
# 행전체가 결측치인 것 중 개별재무 데이터로 대체한 컬럼 말고 나머지 결측치 컬럼 리스트(58개))
list_na_col = ['자산','총자본증가율','비유동자산증가율','유동자산증가율','자기자본증가율',          
'매출액증가율','정상영업이익증가율','순이익증가율','총포괄이익증가율',         
'매출액순이익률','총자본순이익률','자기자본순이익률',         
'매출원가대매출액비율','광고선전비대매출액비율','유보율',              
'비유동비율','유동비율','당좌비율','현금비율','부채비율','유동부채비율',           
'차입금비율','이자보상배율(이자비용)','유보액대비율','유보액대납입자본배율','투자집중도'   ]


df_재무_결측 = df_재무_결측.drop(list_na_col, axis=1)
df_재무_결측 = pd.merge(df_재무_결측, df_재무_개별[["거래소코드","회계년도"]+list_na_col], on=["거래소코드","회계년도"], how = 'left')
# df_재무_결측

df_재무 = pd.concat([df_재무_완전,df_재무_결측], axis=0)

In [29]:
df_재무.isna().sum().to_dict()

{'회사명': 0,
 '거래소코드': 0,
 '회계년도': 0,
 '상장일': 0,
 '자산': 376,
 '당기순이익(손실)': 375,
 '미처분이익잉여금(결손금)': 375,
 '총자본증가율': 448,
 '비유동자산증가율': 448,
 '유동자산증가율': 448,
 '자기자본증가율': 448,
 '매출액증가율': 448,
 '정상영업이익증가율': 448,
 '순이익증가율': 448,
 '총포괄이익증가율': 448,
 '매출액순이익률': 448,
 '총자본순이익률': 448,
 '자기자본순이익률': 448,
 '매출원가대매출액비율': 448,
 '광고선전비대매출액비율': 448,
 '유보율': 448,
 '비유동비율': 448,
 '유동비율': 448,
 '당좌비율': 448,
 '현금비율': 448,
 '부채비율': 448,
 '유동부채비율': 448,
 '차입금비율': 448,
 '이자보상배율(이자비용)': 448,
 '유보액대비율': 448,
 '유보액대납입자본배율': 448,
 '투자집중도': 448}

In [30]:
print("제거행 갯수 : ", len(df_재무[df_재무["유보율"].isna()]))
print("제거기업 갯수 : ", len(list(df_재무[df_재무["유보율"].isna()]["거래소코드"].unique())))
# 약 개 기업의 어떤 년도에는 재무,별도 데이터 둘 다 없는 컬럼이 존재하기에 해당 연도 데이터 제거!(448행)
# 데이터 누락된 사유 : "필수 공시 지표가 아닌 지표는 누락되어 데이터 존재x" 또는 "비상장기업(이었던 기업) 

제거행 갯수 :  448
제거기업 갯수 :  249


In [31]:
df_재무 = df_재무[df_재무["유보율"].isna().map(func_not)] # 위 결측치 없는 데이터로 필터링
df_재무

df_재무.isna().sum().to_dict()

{'회사명': 0,
 '거래소코드': 0,
 '회계년도': 0,
 '상장일': 0,
 '자산': 0,
 '당기순이익(손실)': 0,
 '미처분이익잉여금(결손금)': 0,
 '총자본증가율': 0,
 '비유동자산증가율': 0,
 '유동자산증가율': 0,
 '자기자본증가율': 0,
 '매출액증가율': 0,
 '정상영업이익증가율': 0,
 '순이익증가율': 0,
 '총포괄이익증가율': 0,
 '매출액순이익률': 0,
 '총자본순이익률': 0,
 '자기자본순이익률': 0,
 '매출원가대매출액비율': 0,
 '광고선전비대매출액비율': 0,
 '유보율': 0,
 '비유동비율': 0,
 '유동비율': 0,
 '당좌비율': 0,
 '현금비율': 0,
 '부채비율': 0,
 '유동부채비율': 0,
 '차입금비율': 0,
 '이자보상배율(이자비용)': 0,
 '유보액대비율': 0,
 '유보액대납입자본배율': 0,
 '투자집중도': 0}

In [32]:
df_재무.to_csv('재무완성.csv',encoding='UTF-8-sig')